# Task 1 f, Calculating reserve factors against column buckling using Euler-Johnson

## We need from the data file:
- volume of the element
- Iyy calculated with `height_str, width_str, thickness_web, thickness_flange, thickness_skin, stringer_pitch`
- dimensions
- `EulerJohnson(EModulus, I_y, area, length, height_str, thickness_flange, thickness_web, radius, sigma_yield, sigma_applied, c=1)`

## Please enter your name (daniel, fabian, felix, yannis)

In [1]:
name = 'yannis'

## Imports

In [2]:
import pandas as pd
import numpy as np
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.helpers import *
from optimization.generation import *

In [3]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

## Constants

In [4]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}. Please verify.")

stringer_pitch = 200
effective_width = stringer_pitch/2

panel_element_length = 750/3
stringer_element_length = 750/3


Your personal data is: sigma_yield = 490, EModulus = 65241.07. Please verify.


# Importing necessary files 

## Import everything for panels 

In [5]:
# Import panel properties and match the elements to the respective panel
paneldf = pd.read_csv(f'../data/{name}/panel.csv')
panelPropertiesdf = pd.read_csv(f'../data/{name}/properties/panel_properties.csv', index_col=0)
panelPropertiesdf = panelPropertiesdf.drop(['mass', 'Component Name'], axis=1)
paneldf = pd.merge(paneldf, panelPropertiesdf, on='Element ID', how='left', suffixes=('_caller', '_other'))
paneldf = paneldf.drop(['sigmaXY', 'sigmaYY'], axis=1)
paneldf.head(5)

,Element ID,Component Name,sigmaXX,Load Case,thickness
0,1,panel1,-69.340,Subcase 1 (LC1),4.0
1,2,panel1,-81.340,Subcase 1 (LC1),4.0
2,3,panel1,-70.443,Subcase 1 (LC1),4.0
3,4,panel2,-96.966,Subcase 1 (LC1),4.0
4,5,panel2,-89.099,Subcase 1 (LC1),4.0


## Add a volume column to the panels 

In [6]:
paneldf['Volume'] = paneldf.apply(panel_element_volume, elementLength=panel_element_length, elementWidth=stringer_pitch, axis=1)
paneldf.head(3)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume
0,1,panel1,-69.340,Subcase 1 (LC1),4.0,100000.0
1,2,panel1,-81.340,Subcase 1 (LC1),4.0,100000.0
2,3,panel1,-70.443,Subcase 1 (LC1),4.0,100000.0


## Import everything for stringers 

In [7]:
# Open and match stringer properties 
stringerdf = pd.read_csv(f'../data/{name}/stringer.csv')
# Now add the stringer properties '../data/{name}/stringer_properties.csv
stringerPropertiesdf = pd.read_csv(f'../data/{name}/properties/stringer_properties.csv', index_col=0)
stringerPropertiesdf = stringerPropertiesdf.reset_index()
stringerPropertiesdf.rename(columns={'beamsects': 'Component Name', 'beamsect_dim1': 'dim1', 'beamsect_dim2': 'dim2', 'beamsect_dim3': 'dim3', 'beamsect_dim4': 'dim4'}, inplace=True)
# Add "stringer" prefix to Component Name
stringerPropertiesdf['Component Name'] = 'stringer' + stringerPropertiesdf['Component Name'].astype(str)
# Merge the dataframes
stringerdf = pd.merge(stringerdf, stringerPropertiesdf, on='Component Name', how='left', suffixes=('_caller', '_other'))
stringerdf

,Element ID,Component Name,sigmaXX,Load Case,dim1,dim2,dim3,dim4
0,40,stringer1,-71.096,Subcase 1 (LC1),25.0,2.0,20.0,15.0
1,41,stringer1,-68.223,Subcase 1 (LC1),25.0,2.0,20.0,15.0
2,42,stringer1,-73.429,Subcase 1 (LC1),25.0,2.0,20.0,15.0
3,43,stringer2,-75.797,Subcase 1 (LC1),25.0,2.0,20.0,15.0
4,44,stringer2,-79.653,Subcase 1 (LC1),25.0,2.0,20.0,15.0
...,...,...,...,...,...,...,...,...
76,62,stringer8,-26.462,Subcase 3 (LC3),25.0,2.0,20.0,15.0
77,63,stringer8,-18.253,Subcase 3 (LC3),25.0,2.0,20.0,15.0
78,64,stringer9,-28.657,Subcase 3 (LC3),25.0,2.0,20.0,15.0
79,65,stringer9,-22.665,Subcase 3 (LC3),25.0,2.0,20.0,15.0


## Add volume to the stringer elements

In [8]:
stringerdf['Volume']= stringerdf.apply(stringer_element_volume, elementLength = stringer_element_length, axis=1)
stringerdf.head(3)

,Element ID,Component Name,sigmaXX,Load Case,dim1,dim2,dim3,dim4,Volume
0,40,stringer1,-71.096,Subcase 1 (LC1),25.0,2.0,20.0,15.0,48000.0
1,41,stringer1,-68.223,Subcase 1 (LC1),25.0,2.0,20.0,15.0,48000.0
2,42,stringer1,-73.429,Subcase 1 (LC1),25.0,2.0,20.0,15.0,48000.0


# Now we rearrange the structure a bit

## First we split the 3 loadcases 

In [9]:
loadCase1dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 1 (LC1)']
loadCase2dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 2 (LC2)']
loadCase3dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 3 (LC3)']
loadCase1dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 1 (LC1)']
loadCase2dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 2 (LC2)']
loadCase3dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 3 (LC3)']
print(loadCase1dfPanel.head(5))
print(loadCase1dfStringer.head(5))

   Element ID Component Name  sigmaXX        Load Case  thickness    Volume
0           1         panel1  -69.340  Subcase 1 (LC1)        4.0  100000.0
1           2         panel1  -81.340  Subcase 1 (LC1)        4.0  100000.0
2           3         panel1  -70.443  Subcase 1 (LC1)        4.0  100000.0
3           4         panel2  -96.966  Subcase 1 (LC1)        4.0  100000.0
4           5         panel2  -89.099  Subcase 1 (LC1)        4.0  100000.0
   Element ID Component Name  sigmaXX        Load Case  dim1  dim2  dim3  \
0          40      stringer1  -71.096  Subcase 1 (LC1)  25.0   2.0  20.0   
1          41      stringer1  -68.223  Subcase 1 (LC1)  25.0   2.0  20.0   
2          42      stringer1  -73.429  Subcase 1 (LC1)  25.0   2.0  20.0   
3          43      stringer2  -75.797  Subcase 1 (LC1)  25.0   2.0  20.0   
4          44      stringer2  -79.653  Subcase 1 (LC1)  25.0   2.0  20.0   

   dim4   Volume  
0  15.0  48000.0  
1  15.0  48000.0  
2  15.0  48000.0  
3  15.0  48

# Now we need to combine the panels and the stringers

In [10]:
# For every loadcase
# Efficiently combine panels for load case 1 
panel_groups_1 = []
for i in range(1, 10):
    df1 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase1dfStringer[loadCase1dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_1.extend([df1, df2, df3])
lc1combined = pd.concat(panel_groups_1, ignore_index=True)
lc1combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-69.340,Subcase 1 (LC1),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-81.340,Subcase 1 (LC1),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-70.443,Subcase 1 (LC1),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-96.966,Subcase 1 (LC1),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-89.099,Subcase 1 (LC1),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-95.415,Subcase 1 (LC1),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-71.096,Subcase 1 (LC1),NaN,48000.0,stiffener1,25.0,2.0,20.0,15.0
7,41,stringer1,-68.223,Subcase 1 (LC1),NaN,48000.0,stiffener1,25.0,2.0,20.0,15.0
8,42,stringer1,-73.429,Subcase 1 (LC1),NaN,48000.0,stiffener1,25.0,2.0,20.0,15.0
9,4,panel2,-96.966,Subcase 1 (LC1),4.0,100000.0,stiffener2,NaN,NaN,NaN,NaN


In [11]:
# Efficiently combine panels for load case 2
panel_groups_2 = []
for i in range(1, 10):
    df1 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase2dfStringer[loadCase2dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_2.extend([df1, df2, df3])
lc2combined = pd.concat(panel_groups_2, ignore_index=True)
lc2combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-71.949,Subcase 2 (LC2),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-83.026,Subcase 2 (LC2),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-73.051,Subcase 2 (LC2),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-97.075,Subcase 2 (LC2),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-89.026,Subcase 2 (LC2),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-95.524,Subcase 2 (LC2),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-73.576,Subcase 2 (LC2),NaN,48000.0,stiffener1,25.0,2.0,20.0,15.0
7,41,stringer1,-77.509,Subcase 2 (LC2),NaN,48000.0,stiffener1,25.0,2.0,20.0,15.0
8,42,stringer1,-75.909,Subcase 2 (LC2),NaN,48000.0,stiffener1,25.0,2.0,20.0,15.0
9,4,panel2,-97.075,Subcase 2 (LC2),4.0,100000.0,stiffener2,NaN,NaN,NaN,NaN


In [12]:
# Efficiently combine panels for load case 3 
panel_groups_3 = []
for i in range(1, 10):
    df1 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase3dfStringer[loadCase3dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_3.extend([df1, df2, df3])
lc3combined = pd.concat(panel_groups_3, ignore_index=True)
lc3combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-21.021,Subcase 3 (LC3),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-27.022,Subcase 3 (LC3),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-25.417,Subcase 3 (LC3),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-35.048,Subcase 3 (LC3),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-29.600,Subcase 3 (LC3),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-28.864,Subcase 3 (LC3),4.0,100000.0,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-19.357,Subcase 3 (LC3),NaN,48000.0,stiffener1,25.0,2.0,20.0,15.0
7,41,stringer1,-22.665,Subcase 3 (LC3),NaN,48000.0,stiffener1,25.0,2.0,20.0,15.0
8,42,stringer1,-28.657,Subcase 3 (LC3),NaN,48000.0,stiffener1,25.0,2.0,20.0,15.0
9,4,panel2,-35.048,Subcase 3 (LC3),4.0,100000.0,stiffener2,NaN,NaN,NaN,NaN


In [13]:
# We fill the empty spaces with zero
lc1combined = lc1combined.fillna(0)
lc2combined = lc2combined.fillna(0)
lc3combined = lc3combined.fillna(0)
lc1combined

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-69.340,Subcase 1 (LC1),4.0,100000.0,stiffener1,0.0,0.0,0.0,0.0
1,2,panel1,-81.340,Subcase 1 (LC1),4.0,100000.0,stiffener1,0.0,0.0,0.0,0.0
2,3,panel1,-70.443,Subcase 1 (LC1),4.0,100000.0,stiffener1,0.0,0.0,0.0,0.0
3,4,panel2,-96.966,Subcase 1 (LC1),4.0,100000.0,stiffener1,0.0,0.0,0.0,0.0
4,5,panel2,-89.099,Subcase 1 (LC1),4.0,100000.0,stiffener1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
76,29,panel10,-81.340,Subcase 1 (LC1),4.0,100000.0,stiffener9,0.0,0.0,0.0,0.0
77,30,panel10,-69.340,Subcase 1 (LC1),4.0,100000.0,stiffener9,0.0,0.0,0.0,0.0
78,64,stringer9,-73.429,Subcase 1 (LC1),0.0,48000.0,stiffener9,25.0,2.0,20.0,15.0
79,65,stringer9,-68.223,Subcase 1 (LC1),0.0,48000.0,stiffener9,25.0,2.0,20.0,15.0


# Now we can aggregate the loadcases according to stiffeners

## Multiply volume and stress together for averaging 

In [14]:
lc1combined['XX * Volume'] = lc1combined['sigmaXX'] * lc1combined['Volume']
lc2combined['XX * Volume'] = lc2combined['sigmaXX'] * lc2combined['Volume']
lc3combined['XX * Volume'] = lc3combined['sigmaXX'] * lc3combined['Volume']
lc1combined.head(2)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4,XX * Volume
0,1,panel1,-69.34,Subcase 1 (LC1),4.0,100000.0,stiffener1,0.0,0.0,0.0,0.0,-6934000.0
1,2,panel1,-81.34,Subcase 1 (LC1),4.0,100000.0,stiffener1,0.0,0.0,0.0,0.0,-8134000.0


## Load case 1 

In [15]:
lc1combined = lc1combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'thickness':'max',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc1combined['sigma_XX_avg'] = lc1combined['XX * Volume'] / lc1combined['Volume']
lc1combined = lc1combined.drop(['XX * Volume'], axis=1)
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg
Stiffener,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-81.279844
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-90.999022
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-91.105554
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.511651
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.193892
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.511651
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-91.105554
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-90.999022
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-81.279844


## Load case 2

In [16]:
lc2combined = lc2combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'thickness':'max',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc2combined['sigma_XX_avg'] = lc2combined['XX * Volume'] / lc2combined['Volume']
lc2combined = lc2combined.drop(['XX * Volume'], axis=1)
lc2combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg
Stiffener,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-83.146253
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-90.417430
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-91.067280
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.418806
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.189301
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.418806
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-91.067280
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-90.417430
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-83.146253


## Load case 3

In [17]:
lc3combined = lc3combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'thickness':'max',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc3combined['sigma_XX_avg'] = lc3combined['XX * Volume'] / lc3combined['Volume']
lc3combined = lc3combined.drop(['XX * Volume'], axis=1)
lc3combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg
Stiffener,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-27.002409
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-30.231296
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-30.266715
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-30.069387
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-29.963914
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-30.069387
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-30.266715
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-30.231296
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-27.002409


# Now we add Cross-Section Properties of the combined skin and hat stringer crosssection 

In [18]:
# Load case 1
lc1combined[['I_yy', 'areaTot', 'VolumeTot']] = lc1combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
# Load case 2
lc2combined[['I_yy', 'areaTot', 'VolumeTot']] = lc2combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
# Load case 3
lc3combined[['I_yy', 'areaTot', 'VolumeTot']] = lc3combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot
Stiffener,,,,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-81.279844,43392.340054,992.0,744000.0
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-90.999022,43392.340054,992.0,744000.0
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-91.105554,43392.340054,992.0,744000.0
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.511651,43392.340054,992.0,744000.0
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.193892,43392.340054,992.0,744000.0
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.511651,43392.340054,992.0,744000.0
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-91.105554,43392.340054,992.0,744000.0
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-90.999022,43392.340054,992.0,744000.0
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-81.279844,43392.340054,992.0,744000.0


# Now we calculate the columnbuckling with Euler Johnson 

In [19]:
# Load case 1EModulus, DIM1, DIM2, DIM3, sigma_yield, r
lc1combined['sigma_crip'] = lc1combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
# Load case 2
lc2combined['sigma_crip'] = lc2combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
#Load case 3
lc3combined['sigma_crip'] = lc3combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip
Stiffener,,,,,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-81.279844,43392.340054,992.0,744000.0,528.835519
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-90.999022,43392.340054,992.0,744000.0,528.835519
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-91.105554,43392.340054,992.0,744000.0,528.835519
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.511651,43392.340054,992.0,744000.0,528.835519
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.193892,43392.340054,992.0,744000.0,528.835519
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.511651,43392.340054,992.0,744000.0,528.835519
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-91.105554,43392.340054,992.0,744000.0,528.835519
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-90.999022,43392.340054,992.0,744000.0,528.835519
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-81.279844,43392.340054,992.0,744000.0,528.835519


## Add lambda_crit to the loadcases

In [20]:
lc1combined['lambda_crit'] = lc1combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)
lc2combined['lambda_crit'] = lc2combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)
lc3combined['lambda_crit'] = lc3combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)

## Add lambda to the loadcases

In [21]:
lc1combined['lambda'] = lc1combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc2combined['lambda'] = lc2combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc3combined['lambda'] = lc3combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)

## Add radius of gyration to the loadcases

In [22]:
lc1combined['r_gyr'] = lc1combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc2combined['r_gyr'] = lc2combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc3combined['r_gyr'] = lc3combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)

## Now calculate the critical stress & Reserve Factor

In [23]:
lc1combined[['sigma_crit', 'Reserve Factor']] = lc1combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc2combined[['sigma_crit', 'Reserve Factor']] = lc2combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc3combined[['sigma_crit', 'Reserve Factor']] = lc3combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
Stiffener,,,,,,,,,,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-81.279844,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.410701
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-90.999022,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.366836
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-91.105554,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.366407
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.511651,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.368811
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.193892,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.370110
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.511651,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.368811
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-91.105554,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.366407
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-90.999022,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.366836
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-81.279844,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.410701


# Generate Score output 

In [24]:

lc1combined = lc1combined.reset_index()
lc2combined = lc2combined.reset_index()
lc3combined = lc3combined.reset_index()

evaluateDf = pd.concat([lc1combined, lc2combined, lc3combined])
evaluateDf = evaluateDf.reset_index()
evaluateDf = evaluateDf.drop(['index'], axis=1)
evaluateDf

,Stiffener,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
0,stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-81.279844,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.410701
1,stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-90.999022,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.366836
2,stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-91.105554,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.366407
3,stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.511651,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.368811
4,stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.193892,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.370110
5,stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.511651,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.368811
6,stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-91.105554,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.366407
7,stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-90.999022,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.366836
8,stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-81.279844,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.410701
9,stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-83.146253,43392.340054,992.0,744000.0,528.835519,51.265756,113.39935,6.613795,50.072548,0.401482


In [25]:
evaluateDf = evaluateDf.drop(['Volume' ,'thickness' , 'sigma_XX_avg', 'I_yy',  'areaTot', 'VolumeTot', 'sigma_crip', 'lambda_crit', 'lambda', 'r_gyr', 'sigma_crit'], axis =1)
evaluateDf['score'] = evaluateDf.apply(rf_score, axis=1)
evaluateDf

,Stiffener,dim1,dim2,dim3,dim4,Reserve Factor,score
0,stiffener1,25.0,2.0,20.0,15.0,0.410701,127.859836
1,stiffener2,25.0,2.0,20.0,15.0,0.366836,136.632840
2,stiffener3,25.0,2.0,20.0,15.0,0.366407,136.718630
3,stiffener4,25.0,2.0,20.0,15.0,0.368811,136.237786
4,stiffener5,25.0,2.0,20.0,15.0,0.370110,135.977917
5,stiffener6,25.0,2.0,20.0,15.0,0.368811,136.237786
6,stiffener7,25.0,2.0,20.0,15.0,0.366407,136.718630
7,stiffener8,25.0,2.0,20.0,15.0,0.366836,136.632840
8,stiffener9,25.0,2.0,20.0,15.0,0.410701,127.859836
9,stiffener1,25.0,2.0,20.0,15.0,0.401482,129.703660


## Extract score and parameters

In [26]:
score = evaluateDf['score'].sum()
dimensions = extractDimensions(evaluateDf)

evaluateDf = pd.DataFrame({
    'stringer Parameters': [dimensions],
    'score': [score]
})
evaluateDf

,stringer Parameters,score
0,"[[25.0, 2.0, 20.0, 15.0], [25.0, 2.0, 20.0, 15...",2501.391969


# Cleanup data for output 

## Drop unenessacry columns 

In [27]:
lc1combined = lc1combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc2combined = lc2combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc3combined = lc3combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc1combined

,Stiffener,sigma_XX_avg,I_yy,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
0,stiffener1,-81.279844,43392.340054,528.835519,51.265756,113.39935,6.613795,50.072548,0.410701
1,stiffener2,-90.999022,43392.340054,528.835519,51.265756,113.39935,6.613795,50.072548,0.366836
2,stiffener3,-91.105554,43392.340054,528.835519,51.265756,113.39935,6.613795,50.072548,0.366407
3,stiffener4,-90.511651,43392.340054,528.835519,51.265756,113.39935,6.613795,50.072548,0.368811
4,stiffener5,-90.193892,43392.340054,528.835519,51.265756,113.39935,6.613795,50.072548,0.370110
5,stiffener6,-90.511651,43392.340054,528.835519,51.265756,113.39935,6.613795,50.072548,0.368811
6,stiffener7,-91.105554,43392.340054,528.835519,51.265756,113.39935,6.613795,50.072548,0.366407
7,stiffener8,-90.999022,43392.340054,528.835519,51.265756,113.39935,6.613795,50.072548,0.366836
8,stiffener9,-81.279844,43392.340054,528.835519,51.265756,113.39935,6.613795,50.072548,0.410701


## ROUND & Add together the load cases 

In [28]:
# Rename colums for concat
lc1combined = lc1combined.rename(columns={'sigma_XX_avg':'XX_avg_LC1', 'sigma_crit':'sigma_crit_LC1', 'Reserve Factor':'RF_LC1', 'sigma_crip':'sigma_crip_LC1'})
lc2combined = lc2combined.rename(columns={'sigma_XX_avg':'XX_avg_LC2', 'sigma_crit':'sigma_crit_LC2', 'Reserve Factor':'RF_LC2', 'sigma_crip':'sigma_crip_LC2'})
lc3combined = lc3combined.rename(columns={'sigma_XX_avg':'XX_avg_LC3', 'sigma_crit':'sigma_crit_LC3', 'Reserve Factor':'RF_LC3', 'sigma_crip':'sigma_crip_LC3'})

In [29]:
outputdf = pd.concat([lc1combined,lc2combined,lc3combined], axis = 1)
outputdf = outputdf.round(rounding_digits)
# After concatenation, keep only the first column of cross section propertries and drop the rest
outputdf['I_y'] = outputdf.filter(like='I_yy').iloc[:, 0]  # Take the first I_yy column
outputdf['Lambda'] = outputdf.filter(like='lambda').iloc[:, 1]  # Take the first lambda column
outputdf['Lambda_crit'] = outputdf.filter(like='lambda_crit').iloc[:, 0]  # Take the first I_yy column
outputdf['R_gyr'] = outputdf.filter(like='r_gyr').iloc[:, 0]  # Take the first I_yy column
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('I_yy')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('lambda')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('r_gyr')])
outputdf.head(10)

,Stiffener,XX_avg_LC1,sigma_crip_LC1,sigma_crit_LC1,RF_LC1,Stiffener,XX_avg_LC2,sigma_crip_LC2,sigma_crit_LC2,RF_LC2,Stiffener,XX_avg_LC3,sigma_crip_LC3,sigma_crit_LC3,RF_LC3,I_y,Lambda,Lambda_crit,R_gyr
0,stiffener1,-81.28,528.84,50.07,0.41,stiffener1,-83.15,528.84,50.07,0.40,stiffener1,-27.00,528.84,50.07,1.24,43392.34,113.4,51.27,6.61
1,stiffener2,-91.00,528.84,50.07,0.37,stiffener2,-90.42,528.84,50.07,0.37,stiffener2,-30.23,528.84,50.07,1.10,43392.34,113.4,51.27,6.61
2,stiffener3,-91.11,528.84,50.07,0.37,stiffener3,-91.07,528.84,50.07,0.37,stiffener3,-30.27,528.84,50.07,1.10,43392.34,113.4,51.27,6.61
3,stiffener4,-90.51,528.84,50.07,0.37,stiffener4,-90.42,528.84,50.07,0.37,stiffener4,-30.07,528.84,50.07,1.11,43392.34,113.4,51.27,6.61
4,stiffener5,-90.19,528.84,50.07,0.37,stiffener5,-90.19,528.84,50.07,0.37,stiffener5,-29.96,528.84,50.07,1.11,43392.34,113.4,51.27,6.61
5,stiffener6,-90.51,528.84,50.07,0.37,stiffener6,-90.42,528.84,50.07,0.37,stiffener6,-30.07,528.84,50.07,1.11,43392.34,113.4,51.27,6.61
6,stiffener7,-91.11,528.84,50.07,0.37,stiffener7,-91.07,528.84,50.07,0.37,stiffener7,-30.27,528.84,50.07,1.10,43392.34,113.4,51.27,6.61
7,stiffener8,-91.00,528.84,50.07,0.37,stiffener8,-90.42,528.84,50.07,0.37,stiffener8,-30.23,528.84,50.07,1.10,43392.34,113.4,51.27,6.61
8,stiffener9,-81.28,528.84,50.07,0.41,stiffener9,-83.15,528.84,50.07,0.40,stiffener9,-27.00,528.84,50.07,1.24,43392.34,113.4,51.27,6.61


# Generate output file 

In [30]:
outputdf.to_excel(f'../data/{name}/output/processed_f.xlsx')